In [ ]:
import os
import numpy as np
import pandas as pd
import absl.logging
from nlp_embedding import GloVe, SmallBert, Bert, Word2vec
from nlp_classifier import NaiveBayes, SVM, XGBoost, CNN
from sklearn import preprocessing, metrics
from ast import literal_eval
absl.logging.set_verbosity(absl.logging.ERROR)

In [ ]:
max_words = 200
dataset_name = 'small_balanced'
cnn_epochs = 5

In [ ]:
def train(data_x, data_y, nlp_embedding, nlp_classifier, label_encoder, batch_size, epochs=1, model_dir='models', start_idx=0):
    print('Training...')
    fname = os.path.join(model_dir, dataset_name, f'model_{nlp_embedding.name}_{nlp_classifier.name}')
    data_y_enc = label_encoder.transform(data_y)
    classes = np.unique(data_y_enc)
    
    for epoch in range(epochs):
        print(f'Epoch: {str(epoch + 1)}/{str(epochs)}')
        for i in range(start_idx, data_x.shape[0], batch_size):
            
            if i + batch_size > data_x.shape[0]:
                j = data_x.shape[0]
            else:
                j = i + batch_size
            
            print(f'Processing rows: {i} - {j - 1}')

            embeddings = nlp_embedding.embed_lyrics(data_x[i:j])
            nlp_classifier.partial_fit(embeddings, data_y_enc[i:j], classes=classes)
            nlp_classifier.save(fname)
        start_idx = 0
    
    print('Success!')

In [ ]:
def test(data_x, nlp_embedding, nlp_classifier, label_encoder, batch_size, pred_dir='predictions', start_idx=0):
    print('Testing...')
    fname = os.path.join(pred_dir, dataset_name, f'model_{nlp_embedding.name}_{nlp_classifier.name}.csv')
    predictions_all = []

    if start_idx == 0 and os.path.exists(fname):
        os.remove(fname)
    
    for i in range(start_idx, data_x.shape[0], batch_size):

        if i + batch_size > data_x.shape[0]:
            j = data_x.shape[0]
        else:
            j = i + batch_size
        
        print(f'Processing rows: {i} - {j - 1}')

        embeddings = nlp_embedding.embed_lyrics(data_x[i:j])
        predictions_enc = nlp_classifier.predict(embeddings)
        predictions = label_encoder.inverse_transform(predictions_enc)
        
        predictions_all.extend(predictions)

        pd.DataFrame(predictions.reshape(-1, 1)).to_csv(fname, mode='a', index=False, header=False)
    
    print('Success!')    
    
    return predictions_all

In [ ]:
def train_and_save_results(emb, clf, x_train, y_train, x_test, y_test, dataset_name, le, batch_size = 5000, epochs=1):
    train(x_train, y_train, emb, clf, le, batch_size=batch_size, epochs=epochs)
    y_pred = test(x_test, emb, clf, le, batch_size=batch_size)
    get_results(y_test, y_pred)
    return

In [ ]:
def get_results(y_true, y_pred):
    print('RESULTS:')
    print(f'accuracy = {metrics.accuracy_score(y_true=y_true, y_pred=y_pred)}')
    print(f'balanced accuracy = {metrics.balanced_accuracy_score(y_true=y_true, y_pred=y_pred)}')
    print(f'f1 score = {metrics.f1_score(y_true=y_true, y_pred=y_pred, average="weighted")}')

In [ ]:
def add_normalized_lyrics(data):
    tokens = data.tokens.apply(literal_eval)
    data['normalized_lyrics'] = [' '.join(t) for t in tokens]

In [ ]:
model_dir = os.path.join('models', dataset_name)
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

pred_dir = os.path.join('predictions', dataset_name)
if not os.path.exists(pred_dir):
    os.makedirs(pred_dir)

In [ ]:
train_data = pd.read_csv(f'data/train/{dataset_name}.csv')
test_data = pd.read_csv(f'data/test/{dataset_name}.csv')

In [ ]:
add_normalized_lyrics(train_data)
add_normalized_lyrics(test_data)

In [ ]:
genres = np.unique(train_data.genre)
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(genres)
label_encoder.classes_

In [ ]:
emb_bert = Bert(max_words)

In [ ]:
emb_sm_bert = SmallBert(max_words)

In [ ]:
emb_glove = GloVe(max_words)

In [ ]:
emb_wv = Word2vec(max_words)

## Smaller BERT
### Not normalized lyrics

In [ ]:
clf_nb = NaiveBayes()
train_and_save_results(emb_sm_bert, clf_nb, 
                       train_data.lyrics, train_data.genre, test_data.lyrics, test_data.genre, 
                       dataset_name, label_encoder)

In [ ]:
clf_svm = SVM()
train_and_save_results(emb_sm_bert, clf_svm, 
                       train_data.lyrics, train_data.genre, test_data.lyrics, test_data.genre, 
                       dataset_name, label_encoder)

In [ ]:
clf_xgb = XGBoost(len(label_encoder.classes_))
train_and_save_results(emb_bert, clf_xgb, 
                       train_data.lyrics, train_data.genre, test_data.lyrics, test_data.genre, 
                       dataset_name, label_encoder)

In [ ]:
clf_cnn = CNN(max_words * emb_sm_bert.embedding_size, len(genres), 'adam')
train_and_save_results(emb_sm_bert, clf_cnn,
                       train_data.lyrics, train_data.genre, test_data.lyrics, test_data.genre, 
                       dataset_name, label_encoder, epochs=cnn_epochs)

### Normalized lyrics

In [ ]:
clf_nb_norm = NaiveBayes()
train_and_save_results(emb_sm_bert, clf_nb_norm, 
                       train_data.normalized_lyrics, train_data.genre, test_data.normalized_lyrics, test_data.genre, 
                       dataset_name, label_encoder)

In [ ]:
clf_svm_norm = SVM()
train_and_save_results(emb_sm_bert, clf_svm_norm, 
                       train_data.normalized_lyrics, train_data.genre, test_data.normalized_lyrics, test_data.genre, 
                       dataset_name, label_encoder)

In [ ]:
clf_xgb_norm = XGBoost(len(label_encoder.classes_))
train_and_save_results(emb_sm_bert, clf_xgb_norm, 
                       train_data.normalized_lyrics, train_data.genre, test_data.normalized_lyrics, test_data.genre, 
                       dataset_name, label_encoder)

In [ ]:
clf_cnn_norm = CNN(max_words * emb_sm_bert.embedding_size, len(genres), 'adam')
train_and_save_results(emb_sm_bert, clf_cnn_norm,
                       train_data.normalized_lyrics, train_data.genre, test_data.normalized_lyrics, test_data.genre, 
                       dataset_name, label_encoder, epochs=cnn_epochs)

## BERT

In [ ]:
clf_cnn_b = CNN(max_words * emb_bert.embedding_size, len(genres), 'adam')
train_and_save_results(emb_bert, clf_cnn_b,
                       train_data.lyrics, train_data.genre, test_data.lyrics, test_data.genre, 
                       dataset_name, label_encoder, epochs=cnn_epochs)

In [ ]:
clf_cnn_b_norm = CNN(max_words * emb_bert.embedding_size, len(genres), 'adam')
train_and_save_results(emb_bert, clf_cnn_b_norm,
                       train_data.normalized_lyrics, train_data.genre, test_data.normalized_lyrics, test_data.genre, 
                       dataset_name, label_encoder, epochs=cnn_epochs)

## GloVe

In [ ]:
clf_cnn_g = CNN(max_words * emb_glove.embedding_size, len(genres), 'adam')
train_and_save_results(emb_glove, clf_cnn_g, 
                       train_data.lyrics, train_data.genre, test_data.lyrics, test_data.genre, 
                       dataset_name, label_encoder, epochs=cnn_epochs)

In [ ]:
clf_cnn_g_norm = CNN(max_words * emb_glove.embedding_size, len(genres), 'adam')
train_and_save_results(emb_glove, clf_cnn_g_norm, 
                       train_data.normalized_lyrics, train_data.genre, test_data.normalized_lyrics, test_data.genre, 
                       dataset_name, label_encoder, epochs=cnn_epochs)

## Word2vec

In [ ]:
clf_cnn_w = CNN(max_words * emb_wv.embedding_size, len(genres), 'adam')
train_and_save_results(emb_wv, clf_cnn_w, 
                       train_data.normalized_lyrics, train_data.genre, test_data.normalized_lyrics, test_data.genre, 
                       dataset_name, label_encoder, epochs=cnn_epochs)

In [ ]:
clf_cnn_w_nn = CNN(max_words * emb_wv.embedding_size, len(genres), 'adam')
train_and_save_results(emb_wv, clf_cnn_w_nn, 
                       train_data.lyrics, train_data.genre, test_data.lyrics, test_data.genre, 
                       dataset_name, label_encoder, epochs=cnn_epochs)